# Get Team Data
This notebook pulls team level data to compare and predict winners.

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import datetime

from importlib import reload
import projections
reload(projections)

pd.options.display.max_colwidth = 200
pd.options.display.max_rows, pd.options.display.min_rows = 35, 35

In [3]:
current_season = 2023

# Get year_soup
url = f'https://www.pro-football-reference.com/years/{current_season}/'
page = requests.get(url)
year_soup = BeautifulSoup(page.content, 'html.parser')

# Get def_soup
url = f'https://www.pro-football-reference.com/years/{current_season}/opp.htm'
page = requests.get(url)
def_soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
reload(projections)
data = projections.GetData(year_soup=year_soup)

## Get Team Offensive Data

In [ ]:
# Uses year_soup

## Get Team Defensive Data

In [50]:
def_soup = data.def_soup

reload(projections)
data = projections.GetData(def_soup=def_soup)

In [51]:
ds = data.team_defense_stats()

Team Defense Table
Team Advanced Defense Table
Passing Defense Table
Rushing Defense Table
Kick & Punt Returns Against Table
Kicking Against Table
Punting Against Table
Scoring Defense Table
Conversions Against Table
Drive Averages Against Table


In [53]:
ds[0]

,id,g,points,total_yards,plays_offense,yds_per_play_offense,turnovers,fumbles_lost,first_down,pass_cmp,...,rush_yds,rush_td,rush_yds_per_att,rush_fd,penalties,penalties_yds,pen_fd,score_pct,turnover_pct,exp_pts_def_tot
San Francisco 49ers,sfo,9,143,2741,553,5.0,17,4,167,230,...,748,4,4.0,49,57,475,20,32.3,18.3,30.34
Kansas City Chiefs,kan,9,143,2594,543,4.8,13,8,159,176,...,1010,3,4.5,52,42,355,17,25.5,13.8,30.25
Dallas Cowboys,dal,9,165,2476,519,4.8,14,4,150,151,...,982,7,4.1,51,56,472,25,29.3,14.1,40.58
Cleveland Browns,cle,9,170,2184,482,4.5,15,6,113,131,...,824,9,3.8,37,63,542,14,22.4,12.9,82.57
New York Jets,nyj,9,172,2770,600,4.6,15,6,169,179,...,1246,5,4.3,65,44,381,19,31.8,12.7,36.78
Tampa Bay Buccaneers,tam,9,173,3187,575,5.5,16,8,179,225,...,785,4,3.7,47,58,482,13,36.5,16.7,-11.63
Baltimore Ravens,rav,11,177,3008,713,4.2,15,5,193,241,...,1141,4,4.3,61,71,555,26,29.4,9.5,74.58
Tennessee Titans,oti,9,180,3042,565,5.4,7,4,170,195,...,995,5,3.9,52,58,462,18,42.6,6.4,-47.00
Pittsburgh Steelers,pit,9,182,3417,610,5.6,18,8,183,189,...,1181,5,4.5,57,59,504,20,36.3,17.6,1.17
Green Bay Packers,gnb,9,182,2885,584,4.9,8,3,181,180,...,1197,9,4.2,74,55,529,17,37.9,8.4,-20.95
